In [1]:
import torch
from rxnfp.models import SmilesClassificationModel, SmilesLanguageModelingModel
import pandas as pd
from rxn_yields.data import generate_buchwald_hartwig_rxns
from rdkit import Chem
from rdkit.Chem import DataStructs, AllChem
import numpy as np

print('Success')

This extension has only been tested with simpletransformers==0.34.4


Success


In [2]:
df = pd.read_excel('../Модели/yieldBERT/rxn_yields/data/Buchwald-Hartwig/Dreher_and_Doyle_input_data.xlsx', sheet_name='FullCV_01')
df['rxn'] = generate_buchwald_hartwig_rxns(df)

train_df = df.iloc[:2767][['rxn', 'Output']] 
test_df = df.iloc[2767:][['rxn', 'Output']] #

train_df.columns = ['text', 'labels']
test_df.columns = ['text', 'labels']
mean = train_df.labels.mean()
std = train_df.labels.std()
train_df['labels'] = (train_df['labels'] - mean) / std
test_df['labels'] = (test_df['labels'] - mean) / std
train_df.head()

,text,labels
0,Clc1ccccn1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2...,1.387974
1,Brc1ccccn1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2...,-0.796876
2,CCc1ccc(I)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccc...,-0.827835
3,FC(F)(F)c1ccc(Cl)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd...,-0.464841
4,COc1ccc(Cl)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2cc...,-1.186082


In [3]:
df.head(2)

,Ligand,Additive,Base,Aryl halide,Output,rxn
0,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=C(P([C@@]3(C[...,CC1=CC(C)=NO1,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,ClC1=NC=CC=C1,70.410458,Clc1ccccn1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2...
1,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=C(P([C@@]3(C[...,O=C(OC)C1=CC=NO1,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,BrC1=NC=CC=C1,11.064457,Brc1ccccn1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2...


In [ ]:
model_path = '../Модели/yieldBERT/rxn_yields/trained_models/buchwald_hartwig/FullCV_01_split_2768/checkpoint-1730-epoch-10'

In [61]:
trained_yield_bert = SmilesClassificationModel('bert', model_path,
                                               num_labels=1,
                                               args={"regression": True,
                                                    'config': {"output_hidden_states": True}},
                                               use_cuda=False,
                                              )
                                               #use_cuda=torch.cuda.is_available())

Setting 'max_len_single_sentence' is now deprecated. This value is automatically set up.
Setting 'max_len_sentences_pair' is now deprecated. This value is automatically set up.


In [59]:
yield_predicted = trained_yield_bert.predict(test_df.head(10).text.values)[0]
yield_predicted = yield_predicted * std + mean

yield_true = test_df.head(10).labels.values
yield_true = yield_true * std + mean

for rxn, pred, true in zip(test_df.head(10).text.values, yield_predicted, yield_true):
    # print(rxn)
    print(f"predicted {pred:.1f} | {true:.1f} true yield")
    print()

Setting 'max_len_single_sentence' is now deprecated. This value is automatically set up.
Setting 'max_len_sentences_pair' is now deprecated. This value is automatically set up.


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

predicted 46.3 | 38.1 true yield

predicted 12.1 | 14.8 true yield

predicted 9.0 | 12.2 true yield

predicted 11.6 | 8.3 true yield

predicted 1.9 | 1.1 true yield

predicted 46.0 | 44.4 true yield

predicted 48.8 | 53.4 true yield

predicted 1.6 | 1.7 true yield

predicted 14.9 | 10.2 true yield

predicted -0.3 | 0.0 true yield



In [5]:
test_df.head(10).text.values

array(['Brc1ccccn1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2-c2ccccc2N~1)C(F)(F)F.COc1ccc(OC)c(P(C(C)(C)C)C(C)(C)C)c1-c1c(C(C)C)cc(C(C)C)cc1C(C)C.CN(C)C(=NC(C)(C)C)N(C)C.c1ccc(CN(Cc2ccccc2)c2ccno2)cc1>>Cc1ccc(Nc2ccccn2)cc1',
       'Brc1cccnc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2-c2ccccc2N~1)C(F)(F)F.CC(C)c1cc(C(C)C)c(-c2ccccc2P(C(C)(C)C)C(C)(C)C)c(C(C)C)c1.CCN=P(N=P(N(C)C)(N(C)C)N(C)C)(N(C)C)N(C)C.COC(=O)c1ccno1>>Cc1ccc(Nc2cccnc2)cc1',
       'FC(F)(F)c1ccc(Br)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2-c2ccccc2N~1)C(F)(F)F.COc1ccc(OC)c(P([C@]23C[C@H]4C[C@H](C[C@H](C4)C2)C3)[C@]23C[C@H]4C[C@H](C[C@H](C4)C2)C3)c1-c1c(C(C)C)cc(C(C)C)cc1C(C)C.CCN=P(N=P(N(C)C)(N(C)C)N(C)C)(N(C)C)N(C)C.Fc1cccc(F)c1-c1ccno1>>Cc1ccc(Nc2ccc(C(F)(F)F)cc2)cc1',
       'FC(F)(F)c1ccc(Cl)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2-c2ccccc2N~1)C(F)(F)F.CC(C)c1cc(C(C)C)c(-c2ccccc2P(C2CCCCC2)C2CCCCC2)c(C(C)C)c1.CN1CCCN2CCCN=C12.CCOC(=O)c1cc(OC)no1>>Cc1ccc(Nc2ccc(C(F)(F)F)cc2)cc1',
       'COc1ccc(Cl)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1

In [117]:
import torch

# Создание копии модели
frozen_model = SmilesClassificationModel('bert', model_path,
                                         num_labels=1,
                                         args={"regression": True},
                                         use_cuda=False)
                                         #use_cuda=torch.cuda.is_available())

# Заморозка параметров классификатора
for param in frozen_model.model.classifier.parameters():
    param.requires_grad = False
for param in frozen_model.model.bert.pooler.parameters():
    param.requires_grad = False
for param in frozen_model.model.bert.encoder.layer.parameters():
    param.requires_grad = False

for param in frozen_model.model.bert.embeddings.parameters():
    param.requires_grad = False
# Предсказания для тестовых данных
predictions = frozen_model.predict(test_df.text.values)

# Вывод нескольких строчек
for rxn, pred in zip(test_df.text.values[:10], predictions[:10]):
    print(f"Reaction: {rxn}")
    print(f"Predicted Yield: {pred[0]:.4f}")
    print()

Setting 'max_len_single_sentence' is now deprecated. This value is automatically set up.
Setting 'max_len_sentences_pair' is now deprecated. This value is automatically set up.


  0%|          | 0/1188 [00:00<?, ?it/s]

  0%|          | 0/149 [00:00<?, ?it/s]

Reaction: Brc1ccccn1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2-c2ccccc2N~1)C(F)(F)F.COc1ccc(OC)c(P(C(C)(C)C)C(C)(C)C)c1-c1c(C(C)C)cc(C(C)C)cc1C(C)C.CN(C)C(=NC(C)(C)C)N(C)C.c1ccc(CN(Cc2ccccc2)c2ccno2)cc1>>Cc1ccc(Nc2ccccn2)cc1
Predicted Yield: 0.5017

Reaction: Brc1cccnc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2-c2ccccc2N~1)C(F)(F)F.CC(C)c1cc(C(C)C)c(-c2ccccc2P(C(C)(C)C)C(C)(C)C)c(C(C)C)c1.CCN=P(N=P(N(C)C)(N(C)C)N(C)C)(N(C)C)N(C)C.COC(=O)c1ccno1>>Cc1ccc(Nc2cccnc2)cc1
Predicted Yield: 0.5017



In [146]:
import torch
from transformers import BertTokenizer
torch.set_default_tensor_type(torch.FloatTensor)

frozen_model = SmilesClassificationModel('bert', model_path,
                                         num_labels=1,
                                         args={"regression": True},
                                         use_cuda=False)
# Заморозка всех параметров модели
for param in frozen_model.model.parameters():
    param.requires_grad = False

# Установка модели в режим оценки
frozen_model.model.eval()

# Создание отдельного токенизатора
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
device = torch.device("cpu")

#frozen_model.model = frozen_model.model.to('cpu')
# Теперь, чтобы получить эмбеддинги
embeddings = []

for rxn_text in test_df.text.values[:10]:
    # Токенизация текста
    inputs = tokenizer.encode_plus(rxn_text,
                                   return_tensors="pt",
                                   truncation=True,
                                   padding=True)
    
    # Перемещение токенизированных данных на устройство CPU
    #inputs = {key: value.to(device) for key, value in inputs.items()}

    # Получение вывода модели
    with torch.no_grad():
        outputs = frozen_model.model(**inputs)

    # Взятие вектора представления из последнего скрытого слоя
    last_hidden_states = outputs[0]["last_hidden_state"]
    embedding = last_hidden_states.mean(dim=1).squeeze().numpy()
    embeddings.append(embedding)

# Вывод первого эмбеддинга
print(embeddings[0])

Setting 'max_len_single_sentence' is now deprecated. This value is automatically set up.
Setting 'max_len_sentences_pair' is now deprecated. This value is automatically set up.


IndexError: index out of range in self

In [178]:
def get_embeddings(model, inputs):
    # Токенизируем SMILES-строку
    encoded_inputs = model.tokenizer.encode(inputs)

    # Преобразуем закодированные входы в словарь
    encoded_inputs_dict = {"input_ids": torch.tensor(encoded_inputs).unsqueeze(0).to(model.device)}  # Предполагаем, что первый элемент списка - это input_ids

    # Подаем словарь в модель
    outputs = model.model(**encoded_inputs_dict)

    # Извлекаем эмбеддинги из выходного слоя
    outputs = model.model(**encoded_inputs_dict)
    embeddings = outputs[0]  # Доступ к первому элементу кортежа

    # Возвращаем векторы эмбеддингов
    return embeddings

In [206]:
def get_embeddings(model, inputs):
  """
  Получает эмбеддинги для списка строк.

  Args:
    model: Модель обработки текста.
    inputs: Список строк.

  Returns:
    Массив эмбеддингов.
  """

  embeddings = []
  for input in inputs:
    # Токенизируем строку.
    tokenized_input = model.tokenizer.encode(input)

    # Преобразуем закодированные входы в словарь
    encoded_inputs_dict = {"input_ids": torch.tensor(tokenized_input).unsqueeze(0).to(model.device)}

    # Подаем словарь в модель
    outputs = model.model(**encoded_inputs_dict)

    # Извлекаем эмбеддинги из выходного слоя
    embeddings.append(outputs[0])

  return embeddings

In [215]:
#model = SmilesClassificationModel.from_pretrained("bert-base-uncased")
inputs = test_df.text.values[:10]
embeddings = get_embeddings(frozen_model, inputs)

for embedding in embeddings:
    print(embedding.shape)
print(embeddings.shape())

torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])


AttributeError: 'list' object has no attribute 'shape'

In [209]:
import numpy as np
# Отсоединяем тензор от графа вычислений
embeddings_detached = embeddings.detach()

# Преобразуем отсоединенный тензор в массив NumPy
embeddings_np = np.array(embeddings_detached)

# Выводим числовые значения
print(embeddings_np)

AttributeError: 'list' object has no attribute 'detach'

In [192]:
inputs = "C1=CC=C(C=C1)O"

# Удаляем пробелы из SMILES-строки
inputs = inputs.replace(" ", "")

# Получаем эмбеддинги
embeddings = get_embeddings(frozen_model, inputs)

# Отсоединяем тензор от графа вычислений
embeddings_detached = embeddings.detach()

# Преобразуем отсоединенный тензор в массив NumPy
embeddings_np = np.array(embeddings_detached)

# Выводим числовые значения
print(embeddings_np)

[[0.00893622]]


In [182]:
embeddings[0][0]

tensor(0.0089, grad_fn=<SelectBackward>)

In [199]:
frozen_model = SmilesClassificationModel('bert', model_path,
                                         #freeze_encoder=True,
                                         use_cuda=False)

#model = SmilesClassificationModel(
#   model_type="bert",
#    model_name="bert-base-uncased",
#    model_path=model_path,
#    freeze_encoder=True  # или freeze_all_but_one=True
#)

Setting 'max_len_single_sentence' is now deprecated. This value is automatically set up.
Setting 'max_len_sentences_pair' is now deprecated. This value is automatically set up.


In [166]:
text_data = test_df.text.values.astype(str).tolist()
all_tokens = []  # Список для хранения всех токенов
for text in text_data:
    tokens = tokenizer.tokenize(text)
    all_tokens.extend(tokens)  # Добавляем токены в общий список

input_ids = tokenizer.convert_tokens_to_ids(all_tokens)  # Теперь это список токенов, а не список списков
tensor_inputs = torch.tensor([input_ids])
input_dict = {"input_ids": tensor_inputs}
outputs = frozen_model.forward(**input_dict)  # Используем метод forward

AttributeError: 'SmilesClassificationModel' object has no attribute 'forward'

In [ ]:
last_hidden_state = outputs.last_hidden_state
vector_representations = last_hidden_state[:, 0, :]  # Извлечь представления из первого токена

In [152]:
embeddings

[]

In [130]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [116]:
for name, param in frozen_model.model.named_parameters():
    print(f"Layer name: {name}, Requires gradient: {param.requires_grad}")

Layer name: bert.embeddings.word_embeddings.weight, Requires gradient: False
Layer name: bert.embeddings.position_embeddings.weight, Requires gradient: False
Layer name: bert.embeddings.token_type_embeddings.weight, Requires gradient: False
Layer name: bert.embeddings.LayerNorm.weight, Requires gradient: False
Layer name: bert.embeddings.LayerNorm.bias, Requires gradient: False
Layer name: bert.encoder.layer.0.attention.self.query.weight, Requires gradient: False
Layer name: bert.encoder.layer.0.attention.self.query.bias, Requires gradient: False
Layer name: bert.encoder.layer.0.attention.self.key.weight, Requires gradient: False
Layer name: bert.encoder.layer.0.attention.self.key.bias, Requires gradient: False
Layer name: bert.encoder.layer.0.attention.self.value.weight, Requires gradient: False
Layer name: bert.encoder.layer.0.attention.self.value.bias, Requires gradient: False
Layer name: bert.encoder.layer.0.attention.output.dense.weight, Requires gradient: False
Layer name: bert.e

In [114]:
print(len(test_df.text.values))
print(len(predictions[1]))
print(predictions[0][0])
print(predictions[1][0])
print(predictions[0][1])
print(predictions[1][1])

1188
1188
0.5017356
0.5017356
-0.7569299
-0.7569299


In [81]:
import torch

# Создание копии модели
frozen_model = SmilesClassificationModel('bert', model_path,
                                         num_labels=1,
                                         args={"regression": True},
                                         use_cuda=torch.cuda.is_available())

# Заморозка параметров классификатора
for param in frozen_model.model.classifier.parameters():
    param.requires_grad = False

# Предсказания для тестовых данных
embeddings = []

for text in test_df.text.values:
    # Получение внутреннего представления из последнего скрытого слоя
    with torch.no_grad():
        input_ids = frozen_model.tokenizer.encode(text, return_tensors="pt")
        outputs = frozen_model.model(input_ids)

    # Взятие вектора представления из последнего скрытого слоя
    last_hidden_states = outputs[0].mean(dim=1).squeeze().numpy()
    embeddings.append(last_hidden_states)

# embeddings теперь содержит внутренние представления для каждой реакции


Setting 'max_len_single_sentence' is now deprecated. This value is automatically set up.
Setting 'max_len_sentences_pair' is now deprecated. This value is automatically set up.


In [28]:
for rxn, pred in zip(test_df.text.values[:10], predictions[:10]):
    print(f"Reaction: {rxn}")
    print(f"Predicted Yield: {pred[0].item():.4f}")
    print()

Reaction: Brc1ccccn1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2-c2ccccc2N~1)C(F)(F)F.COc1ccc(OC)c(P(C(C)(C)C)C(C)(C)C)c1-c1c(C(C)C)cc(C(C)C)cc1C(C)C.CN(C)C(=NC(C)(C)C)N(C)C.c1ccc(CN(Cc2ccccc2)c2ccno2)cc1>>Cc1ccc(Nc2ccccn2)cc1
Predicted Yield: 0.5017

Reaction: Brc1cccnc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2-c2ccccc2N~1)C(F)(F)F.CC(C)c1cc(C(C)C)c(-c2ccccc2P(C(C)(C)C)C(C)(C)C)c(C(C)C)c1.CCN=P(N=P(N(C)C)(N(C)C)N(C)C)(N(C)C)N(C)C.COC(=O)c1ccno1>>Cc1ccc(Nc2cccnc2)cc1
Predicted Yield: 0.5017



In [100]:
for name, param in trained_yield_bert.model.named_parameters():
    print(f"Layer name: {name}, Size: {param.size()}")

Layer name: bert.embeddings.word_embeddings.weight, Size: torch.Size([591, 256])
Layer name: bert.embeddings.position_embeddings.weight, Size: torch.Size([512, 256])
Layer name: bert.embeddings.token_type_embeddings.weight, Size: torch.Size([2, 256])
Layer name: bert.embeddings.LayerNorm.weight, Size: torch.Size([256])
Layer name: bert.embeddings.LayerNorm.bias, Size: torch.Size([256])
Layer name: bert.encoder.layer.0.attention.self.query.weight, Size: torch.Size([256, 256])
Layer name: bert.encoder.layer.0.attention.self.query.bias, Size: torch.Size([256])
Layer name: bert.encoder.layer.0.attention.self.key.weight, Size: torch.Size([256, 256])
Layer name: bert.encoder.layer.0.attention.self.key.bias, Size: torch.Size([256])
Layer name: bert.encoder.layer.0.attention.self.value.weight, Size: torch.Size([256, 256])
Layer name: bert.encoder.layer.0.attention.self.value.bias, Size: torch.Size([256])
Layer name: bert.encoder.layer.0.attention.output.dense.weight, Size: torch.Size([256, 256

In [101]:
# Заморозка параметров слоя классификатора
for name, param in trained_yield_bert.model.named_parameters():
    if 'classifier' in name:  # Проверка, что это параметры классификатора
        param.requires_grad = False
    if 'pooler' in name:  # Проверка, что это параметры классификатора
        param.requires_grad = False

In [102]:
for name, param in trained_yield_bert.model.named_parameters():
    print(f'Parameter: {name}, Requires gradient: {param.requires_grad}')

Parameter: bert.embeddings.word_embeddings.weight, Requires gradient: True
Parameter: bert.embeddings.position_embeddings.weight, Requires gradient: True
Parameter: bert.embeddings.token_type_embeddings.weight, Requires gradient: True
Parameter: bert.embeddings.LayerNorm.weight, Requires gradient: True
Parameter: bert.embeddings.LayerNorm.bias, Requires gradient: True
Parameter: bert.encoder.layer.0.attention.self.query.weight, Requires gradient: True
Parameter: bert.encoder.layer.0.attention.self.query.bias, Requires gradient: True
Parameter: bert.encoder.layer.0.attention.self.key.weight, Requires gradient: True
Parameter: bert.encoder.layer.0.attention.self.key.bias, Requires gradient: True
Parameter: bert.encoder.layer.0.attention.self.value.weight, Requires gradient: True
Parameter: bert.encoder.layer.0.attention.self.value.bias, Requires gradient: True
Parameter: bert.encoder.layer.0.attention.output.dense.weight, Requires gradient: True
Parameter: bert.encoder.layer.0.attention.

In [6]:
type(rxn)

str

In [7]:
# trained_yield_bert.predict(smiles)

### Далее попробуем достать предикты в векторном представлении

In [8]:
def smiles_to_rdkit_mol(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return mol

# Получение векторного представления из молекулярного объекта RDKit
def mol_to_vector(mol):
    # Дескрипторы молекулы
    descriptors = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
    # Преобразование дескрипторов в numpy массив
    array = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(descriptors, array)
    
    return array

In [30]:
# Преобразование реакции в векторное представление
rxns = test_df.head(10).text.values
vectors = []

for rxn in rxns:
    # Разбивка реакции на отдельные молекулы
    reactants, _, products = rxn.split('>')
    reactants = reactants.split('.')
    products = products.split('.')
    
    # Преобразование каждой молекулы в вектор и добавление в список
    reactant_vectors = [mol_to_vector(smiles_to_rdkit_mol(reactant)) for reactant in reactants]
    product_vectors = [mol_to_vector(smiles_to_rdkit_mol(product)) for product in products]
    
    vectors.extend(reactant_vectors + product_vectors)

In [27]:
for rxn, vector in zip(test_df.head(10).text.values, vectors):
    print(rxn)
    print(f'Векторное представление: {vector}')
    print()

Brc1ccccn1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2-c2ccccc2N~1)C(F)(F)F.COc1ccc(OC)c(P(C(C)(C)C)C(C)(C)C)c1-c1c(C(C)C)cc(C(C)C)cc1C(C)C.CN(C)C(=NC(C)(C)C)N(C)C.c1ccc(CN(Cc2ccccc2)c2ccno2)cc1>>Cc1ccc(Nc2ccccn2)cc1
Векторное представление: [0. 0. 0. ... 0. 0. 0.]

Brc1cccnc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2-c2ccccc2N~1)C(F)(F)F.CC(C)c1cc(C(C)C)c(-c2ccccc2P(C(C)(C)C)C(C)(C)C)c(C(C)C)c1.CCN=P(N=P(N(C)C)(N(C)C)N(C)C)(N(C)C)N(C)C.COC(=O)c1ccno1>>Cc1ccc(Nc2cccnc2)cc1
Векторное представление: [0. 0. 0. ... 0. 0. 0.]

FC(F)(F)c1ccc(Br)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2-c2ccccc2N~1)C(F)(F)F.COc1ccc(OC)c(P([C@]23C[C@H]4C[C@H](C[C@H](C4)C2)C3)[C@]23C[C@H]4C[C@H](C[C@H](C4)C2)C3)c1-c1c(C(C)C)cc(C(C)C)cc1C(C)C.CCN=P(N=P(N(C)C)(N(C)C)N(C)C)(N(C)C)N(C)C.Fc1cccc(F)c1-c1ccno1>>Cc1ccc(Nc2ccc(C(F)(F)F)cc2)cc1
Векторное представление: [0. 0. 0. ... 0. 0. 0.]

FC(F)(F)c1ccc(Cl)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2-c2ccccc2N~1)C(F)(F)F.CC(C)c1cc(C(C)C)c(-c2ccccc2P(C2CCCCC2)C2CCCCC2)c(C(C)C)c1.CN1CCCN2C

**На примере последнего вектора постараемся оценить результат**

In [25]:
print(f'Получился вектор длиной {len(vector)}')
print(f'Сумма значений {vector.sum()}')

unique_values = np.unique(vector)
num_unique_values = len(unique_values)
print(f'Содержит {num_unique_values} уникальных значения, а именно {unique_values[0]} и {unique_values[1]}')

Получился вектор длиной 2048
Сумма значений 34.0
Содержит 2 уникальных значения, а именно 0.0 и 1.0


**Следовательно, вытащенный вектор состоит из 34 единиц и 2014 нулей**

In [31]:
#from simpletransformers.classification import ClassificationArgs, SmilesClassificationModel

model = SmilesClassificationModel('bert', model_path,
                                         num_labels=1,
                                         args={"regression": True},
                                         use_cuda=False)

# Пример строки SMILES
smiles_inputs = ["CCO", "CCN"]

# Токенизация и кодирование строк SMILES
encoded_inputs = model.tokenizer.encode(smiles_inputs, return_tensors="pt", padding=True, truncation=True)

# Инференс
outputs = model.model(**encoded_inputs.data)
embeddings = outputs.last_hidden_state.mean(dim=1)  # Предполагается использование среднего пулинга эмбеддингов

print(embeddings)


Setting 'max_len_single_sentence' is now deprecated. This value is automatically set up.
Setting 'max_len_sentences_pair' is now deprecated. This value is automatically set up.


TypeError: BertForSequenceClassification object argument after ** must be a mapping, not Tensor

In [17]:
encoded_inputs

tensor([[12, 11, 11, 13]])

In [16]:
numpy_data = encoded_inputs.numpy()

# Создаем словарь
dict_data = {'your_key': numpy_data}

print(dict_data)

{'your_key': array([[12, 11, 11, 13]])}


In [29]:
dict_data

{'your_key': array([[12, 11, 11, 13]])}

In [35]:
from simpletransformers.classification import ClassificationModel
import torch

# Загрузка модели
model = ClassificationModel(
    "bert",
    'bert-base-uncased',
    num_labels=2,
    args={"num_train_epochs": 1},
)

# Пример входных данных
text = ["Some text here.", "Another text example."]


TypeError: 'BertTokenizer' object is not callable

In [46]:
# Получение выходов (эмбеддингов) для входных данных
outputs = model.model(model.tokenizer.encode(text, return_tensors="pt"))
embeddings = outputs[0]

print(embeddings)

tensor([[-0.0153,  0.4635]], grad_fn=<AddmmBackward>)


In [44]:
**outputs

SyntaxError: invalid syntax (<ipython-input-44-ced6170a3842>, line 1)

In [128]:
model_path = '../Модели/yieldBERT/rxn_yields/trained_models/buchwald_hartwig/FullCV_01_split_2768/checkpoint-1730-epoch-10'

trained_yield_bert = SmilesClassificationModel('bert', model_path,
                                               num_labels=1,
                                               args={"regression": True,
                                                     'config': {"output_hidden_states": True}},
                                               use_cuda=False)

yield_predicted = trained_yield_bert.predict(test_df.head(10).text.values)[0]
#yield_predicted = yield_predicted * std + mean

yield_true = test_df.head(10).labels.values
#yield_true = yield_true * std + mean

for rxn, pred, true in zip(test_df.head(10).text.values, yield_predicted, yield_true):
    print(rxn)
    print(f"predicted {pred:.1f} | {true:.1f} true yield")
    print()

Setting 'max_len_single_sentence' is now deprecated. This value is automatically set up.
Setting 'max_len_sentences_pair' is now deprecated. This value is automatically set up.


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Brc1ccccn1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2-c2ccccc2N~1)C(F)(F)F.COc1ccc(OC)c(P(C(C)(C)C)C(C)(C)C)c1-c1c(C(C)C)cc(C(C)C)cc1C(C)C.CN(C)C(=NC(C)(C)C)N(C)C.c1ccc(CN(Cc2ccccc2)c2ccno2)cc1>>Cc1ccc(Nc2ccccn2)cc1
predicted 0.5 | 0.2 true yield

Brc1cccnc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2-c2ccccc2N~1)C(F)(F)F.CC(C)c1cc(C(C)C)c(-c2ccccc2P(C(C)(C)C)C(C)(C)C)c(C(C)C)c1.CCN=P(N=P(N(C)C)(N(C)C)N(C)C)(N(C)C)N(C)C.COC(=O)c1ccno1>>Cc1ccc(Nc2cccnc2)cc1
predicted -0.8 | -0.7 true yield

FC(F)(F)c1ccc(Br)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2-c2ccccc2N~1)C(F)(F)F.COc1ccc(OC)c(P([C@]23C[C@H]4C[C@H](C[C@H](C4)C2)C3)[C@]23C[C@H]4C[C@H](C[C@H](C4)C2)C3)c1-c1c(C(C)C)cc(C(C)C)cc1C(C)C.CCN=P(N=P(N(C)C)(N(C)C)N(C)C)(N(C)C)N(C)C.Fc1cccc(F)c1-c1ccno1>>Cc1ccc(Nc2ccc(C(F)(F)F)cc2)cc1
predicted -0.9 | -0.8 true yield

FC(F)(F)c1ccc(Cl)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2-c2ccccc2N~1)C(F)(F)F.CC(C)c1cc(C(C)C)c(-c2ccccc2P(C2CCCCC2)C2CCCCC2)c(C(C)C)c1.CN1CCCN2CCCN=C12.CCOC(=O)c1cc(OC)no1>>Cc1ccc(Nc2ccc(C(F)(F)

In [107]:
from transformers import AutoTokenizer, AutoModel

In [109]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

/home/vyacheslav/anaconda/anaconda3/envs/yields/lib/python3.6/site-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,


In [111]:
inputs = tokenizer.encode(test_df.iloc[0]["text"], return_tensors="pt")

In [113]:
inputs

tensor([[ 12,  11,  24,  11,  17,  25,  18,  11,  24,  44,  22,  42,  17,  22,
          44,  18,  17,  44,  11,  11,  11,  11,  31,  11,  30,  20,  18,  15,
          17,  11,  18,  17,  11,  18,  11,  24,  11,  17,  11,  18,  15,  17,
         206,  17,  15,  17,  15,  18,  17,  15,  18,  15,  18,  15,  17,  15,
          18,  17,  15,  18,  15,  18,  11,  31,  11,  17,  15,  17,  15,  18,
          15,  18,  11,  17,  15,  17,  15,  18,  15,  18,  11,  17,  15,  18,
          15,  24,  11,  17,  15,  18,  15,  17,  22,  11,  17,  15,  18,  17,
          15,  18,  15,  18,  25,  17,  15,  18,  15,  24,  11,  17,  11,  17,
          11,  18,  11,  18,  11,  11,  11,  11,  17,  11,  18,  11,  13]])

In [112]:
outputs = model(**inputs)

TypeError: ClassificationModel object argument after ** must be a mapping, not Tensor

In [126]:
import torch
from simpletransformers.classification import ClassificationModel

# Инициализация ClassificationModel
model = ClassificationModel("bert",
                            "bert-base-uncased",
                            use_cuda=False)

In [129]:
results, model_outputs, _ = trained_yield_bert.predict(test_df.head(10).text.values)

# Здесь должны бы вывестить эмбеддинги
print(torch.tensor(model_outputs))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

tensor([ 0.5017, -0.7569, -0.8730, -0.7781, -1.1353,  0.4880,  0.5920, -1.1446,
        -0.6555, -1.2141])


In [137]:
# Пример строки SMILES
smiles_inputs = ["CCO", "CCN"]

# Токенизация строк SMILES
smiles_tokenizer = SmilesTokenizer("smiles")
encoded_inputs = smiles_tokenizer.encode(smiles_inputs, return_tensors="pt", padding=True, truncation=True)

# Инициализация SmilesLanguageModelingModel
smiles_lm_model = SmilesLanguageModelingModel(
    model_type="bert",
    model_name="bert-base-uncased",
    train_files=None,  # Для SmilesLanguageModelingModel не требуется обучение
    args=None,  # Вы можете передать аргументы по своему усмотрению
    use_cuda=False,
)

# Инференс
outputs = smiles_lm_model.model(**encoded_inputs)
embeddings = outputs.last_hidden_state.mean(dim=1)  # Предполагается использование среднего пулинга эмбеддингов

print(embeddings)

ValueError: Can't find a vocabulary file at path 'smiles'. To load the vocabulary from a Google pretrained model use `tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)`

In [149]:
# Загрузка предварительно обученной модели
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = SmilesClassificationModel('bert', model_path, num_labels=1, args={"regression": True}, use_cuda=True)

# Пример строки SMILES
smiles_inputs = ["CCO", "CCN"]

# Инференс
result, outputs = model.predict(smiles_inputs)

Setting 'max_len_single_sentence' is now deprecated. This value is automatically set up.
Setting 'max_len_sentences_pair' is now deprecated. This value is automatically set up.


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [151]:
result

array([0.19940239, 0.30185756], dtype=float32)

In [150]:
# Эмбеддинги
embeddings = outputs  # предполагается использование среднего пулинга эмбеддингов

print(embeddings)

[0.19940239 0.30185756]


In [100]:
trained_yield_bert.args

{'sliding_window': False,
 'tie_value': 1,
 'stride': 0.8,
 'regression': True,
 'lazy_text_column': 0,
 'lazy_text_a_column': None,
 'lazy_text_b_column': None,
 'lazy_labels_column': 1,
 'lazy_header_row': True,
 'lazy_delimiter': '\t',
 'adam_epsilon': 1e-08,
 'best_model_dir': 'outputs/best_model',
 'cache_dir': 'cache_dir/',
 'config': {'output_hidden_states': True},
 'do_lower_case': False,
 'early_stopping_consider_epochs': False,
 'early_stopping_delta': 0,
 'early_stopping_metric': 'eval_loss',
 'early_stopping_metric_minimize': True,
 'early_stopping_patience': 3,
 'encoding': None,
 'eval_batch_size': 8,
 'evaluate_during_training': True,
 'evaluate_during_training_silent': True,
 'evaluate_during_training_steps': 2000,
 'evaluate_during_training_verbose': False,
 'fp16': False,
 'fp16_opt_level': 'O1',
 'gradient_accumulation_steps': 1,
 'learning_rate': 9.659e-05,
 'local_rank': -1,
 'logging_steps': 50,
 'manual_seed': 42,
 'max_grad_norm': 1.0,
 'max_seq_length': 300,
 '

In [74]:
from simpletransformers.classification import ClassificationArgs


model_args = ClassificationArgs()
model_args.config = {"output_hidden_states": True}

ImportError: cannot import name 'ClassificationArgs'

In [82]:
for i in range(10):
    outputs = trained_yield_bert.model(trained_yield_bert.tokenizer.encode(test_df.iloc[i]["text"], return_tensors="pt"))
    embeddings = outputs[0]
    print(embeddings)

tensor([[0.5017]], grad_fn=<AddmmBackward>)
tensor([[-0.7569]], grad_fn=<AddmmBackward>)
tensor([[-0.8730]], grad_fn=<AddmmBackward>)
tensor([[-0.7781]], grad_fn=<AddmmBackward>)
tensor([[-1.1353]], grad_fn=<AddmmBackward>)
tensor([[0.4880]], grad_fn=<AddmmBackward>)
tensor([[0.5920]], grad_fn=<AddmmBackward>)
tensor([[-1.1446]], grad_fn=<AddmmBackward>)
tensor([[-0.6555]], grad_fn=<AddmmBackward>)
tensor([[-1.2141]], grad_fn=<AddmmBackward>)


In [85]:
outputs.count(10)

0

In [53]:
outputs = trained_yield_bert.model(trained_yield_bert.tokenizer.encode(test_df.head(10).text.values, return_tensors="pt"))
embeddings = outputs[0]

print(embeddings)

ValueError: Input is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.

In [63]:
test_df.iloc[2]["text"]

'FC(F)(F)c1ccc(Br)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2-c2ccccc2N~1)C(F)(F)F.COc1ccc(OC)c(P([C@]23C[C@H]4C[C@H](C[C@H](C4)C2)C3)[C@]23C[C@H]4C[C@H](C[C@H](C4)C2)C3)c1-c1c(C(C)C)cc(C(C)C)cc1C(C)C.CCN=P(N=P(N(C)C)(N(C)C)N(C)C)(N(C)C)N(C)C.Fc1cccc(F)c1-c1ccno1>>Cc1ccc(Nc2ccc(C(F)(F)F)cc2)cc1'

In [20]:
!pip install deepchem
!pip install tensorflow

     |████████████████████████████████| 458.3 MB 29 kB/s  eta 0:00:015
     |████████████████████████████████| 1.3 MB 15.8 MB/s eta 0:00:01
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
     |████████████████████████████████| 132 kB 21.2 MB/s eta 0:00:01
     |████████████████████████████████| 4.6 MB 163 kB/s eta 0:00:011
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     |████████████████████████████████| 42 kB 2.2 MB/s  eta 0:00:01
     |████████████████████████████████| 5.6 MB 162 kB/s eta 0:00:011
     |████████████████████████████████| 462 kB 17.1 MB/s eta 0:00:01
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
     |████████████████████████████████| 4.0 MB 162 kB/s eta 0:00:011
     |████████████████████████████████| 289 kB 16.5 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 7.6 MB/s  eta 0:00:01
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
     |████

In [38]:
from transformers import AutoTokenizer, AutoModel
from deepchem.feat.smiles_tokenizer import SmilesTokenizer

In [82]:
tokenizer1 = AutoTokenizer.from_pretrained(model_path)

/home/vyacheslav/anaconda/anaconda3/envs/yields/lib/python3.6/site-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,


In [110]:
tokenizer = SmilesTokenizer

In [158]:
model = SmilesClassificationModel(
    model_type="bert",
    model_name=model_path,  # Или другое имя предварительно обученной модели BERT
    #tokenizer_type=tokenizer,
    #tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME),
    use_cuda=torch.cuda.is_available())

Setting 'max_len_single_sentence' is now deprecated. This value is automatically set up.
Setting 'max_len_sentences_pair' is now deprecated. This value is automatically set up.


In [159]:
outputs = model(tokens)
embeddings = outputs[0]
embeddings

TypeError: 'SmilesClassificationModel' object is not callable

In [160]:
tokens['input_ids']

tensor([[ 12,  11,  11,  11,  24,  11,  17,  25,  18,  11,  24,  44,  22,  42,
          17,  22,  44,  18,  17,  44,  11,  11,  11,  11,  31,  11,  30,  20,
          18,  15,  17,  11,  18,  17,  11,  18,  11,  24,  11,  17,  11,  18,
          15,  17, 206,  17,  15,  17,  15,  18,  17,  15,  18,  15,  18,  15,
          17,  15,  18,  17,  15,  18,  15,  18,  11,  31,  11,  17,  15,  17,
          15,  18,  15,  18,  11,  17,  15,  17,  15,  18,  15,  18,  11,  17,
          15,  18,  15,  24,  11,  17,  15,  18,  15,  17,  22,  11,  17,  15,
          18,  17,  15,  18,  15,  18,  25,  17,  15,  18,  15,  24,  11,  17,
          11,  17,  11,  18,  11,  18,  11,  11,  11,  11,  17,  11,  18,  11,
          11,  11,  13]])

In [163]:
for param in model.model.parameters():
    param.requires_grad = False

# Размораживаем эмбеддинги
for param in model.model.bert.embeddings.parameters():
    param.requires_grad = True

In [168]:
# Токенизация SMILES строки
tokens = tokenizer1.encode_plus(str(test_df.head(2).text.values),
                           return_tensors="pt")

# Получение эмбеддингов
with torch.no_grad():
    outputs = model.model(tokens['input_ids'])
    embeddings = outputs[0]

In [169]:
outputs[0].numpy()

array([[-0.03312767]], dtype=float32)

In [170]:
embeddings

tensor([[-0.0331]])

In [ ]:
for param in frozen_model.model.classifier.parameters():
    param.requires_grad = False
for param in frozen_model.model.bert.pooler.parameters():
    param.requires_grad = False
for param in frozen_model.model.bert.encoder.layer.parameters():
    param.requires_grad = False

for param in frozen_model.model.bert.embeddings.parameters():
    param.requires_grad = False

In [151]:
outputs

(tensor([[0.0111]]),)

In [136]:
model.model

AttributeError: 'BertModel' object has no attribute 'model'

In [127]:
print(model.model)
print(model.config)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(591, 256, padding_idx=0)
      (position_embeddings): Embedding(512, 256)
      (token_type_embeddings): Embedding(2, 256)
      (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.7987, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=256, out_features=256, bias=True)
              (LayerNorm): LayerNorm((256,), eps=1e-12, elemen

In [164]:
model.model.named_parameters

<bound method Module.named_parameters of BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(591, 256, padding_idx=0)
      (position_embeddings): Embedding(512, 256)
      (token_type_embeddings): Embedding(2, 256)
      (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.7987, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=256, out_features=256, bias=True)
              (LayerN

## Получение эмбеддингов с классической модели

In [128]:
from transformers import AutoTokenizer, AutoModel
import torch

# Инициализация токенизатора и модели
#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenizer = SmilesTokenizer
model = AutoModel.from_pretrained("bert-base-uncased")

# Пример SMILES строки
smiles_string = "CC(=O)OC1=CC=CC=C1C(=O)O"

In [131]:
# Токенизация SMILES строки
tokens = tokenizer.encode(str(test_df.head(1).text.values),
                          return_tensors="pt")

# Получение эмбеддингов
with torch.no_grad():
    outputs = model(tokens)
    embeddings = outputs[0]

TypeError: encode() missing 1 required positional argument: 'text'

In [108]:
tokens

tensor([[  101,  1031,  1005,  7987,  2278,  2487,  9468,  9468,  2078,  2487,
          1012, 10507,  2487,  9468,  2278,  1006,  1050,  1007, 10507,  2487,
          1012,  1051,  1027,  1055,  1006,  1027,  1051,  1007,  1006,  1051,
          1031, 22851,  1033,  1015,  2278,  2475,  9468,  9468,  2278,  2475,
          1011, 29248,  9468,  9468,  2278,  2475,  2078,  1066,  1015,  1007,
          1039,  1006,  1042,  1007,  1006,  1042,  1007,  1042,  1012,  2522,
          2278,  2487,  9468,  2278,  1006,  1051,  2278,  1007,  1039,  1006,
          1052,  1006,  1039,  1006,  1039,  1007,  1006,  1039,  1007,  1039,
          1007,  1039,  1006,  1039,  1007,  1006,  1039,  1007,  1039,  1007,
         27723,  1011, 27723,  2278,  1006,  1039,  1006,  1039,  1007,  1039,
          1007, 10507,  1006,  1039,  1006,  1039,  1007,  1039,  1007, 10507,
          2487,  2278,  1006,  1039,  1007,  1039,  1012, 27166,  1006,  1039,
          1007,  1039,  1006,  1027, 13316,  1006,  

In [109]:
embeddings

tensor([[[-0.4294, -0.0930, -0.2599,  ..., -0.3932,  0.6979,  0.8974],
         [ 0.3636,  0.5580,  0.0487,  ...,  0.5626,  0.3386,  0.0577],
         [ 0.5226,  0.3864,  0.9493,  ...,  0.1434,  0.1315,  0.0573],
         ...,
         [ 0.7180,  0.4761,  1.1916,  ...,  0.1442,  0.1879, -0.4327],
         [ 0.3096,  0.1615,  0.9234,  ...,  0.2996,  0.2408, -0.0898],
         [ 0.1719, -0.1700,  0.4720,  ...,  0.4140, -0.2088,  0.4093]]])

In [20]:
from rxnfp.tokenization import SmilesTokenizer
from transformers import BertModel


In [39]:
tokenizer1 = AutoTokenizer.from_pretrained(model_path)

/home/vyacheslav/anaconda/anaconda3/envs/yields/lib/python3.6/site-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,


In [48]:
bert_inputs = tokenizer1.encode_plus(text=str(test_df.head(1).labels.values),
                                    max_length=trained_yield_bert.config.max_position_embeddings,
                                    padding=True,
                                    truncation=True,
                                    return_tensors='pt')
                                    #self=tokenizer)

In [49]:
with torch.no_grad():
    output = trained_yield_bert.model(**bert_inputs)

In [50]:
embeddings = output['last_hidden_state'].squeeze()[0].cpu().numpy().tolist()

TypeError: tuple indices must be integers or slices, not str

In [51]:
output

(tensor([[0.4775]]),)

In [128]:
print(trained_yield_bert.model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(591, 256, padding_idx=0)
      (position_embeddings): Embedding(512, 256)
      (token_type_embeddings): Embedding(2, 256)
      (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.7987, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=256, out_features=256, bias=True)
              (LayerNorm): LayerNorm((256,), eps=1e-12, elemen

In [131]:
trained_yield_bert.args

{'sliding_window': False,
 'tie_value': 1,
 'stride': 0.8,
 'regression': True,
 'lazy_text_column': 0,
 'lazy_text_a_column': None,
 'lazy_text_b_column': None,
 'lazy_labels_column': 1,
 'lazy_header_row': True,
 'lazy_delimiter': '\t',
 'adam_epsilon': 1e-08,
 'best_model_dir': 'outputs/best_model',
 'cache_dir': 'cache_dir/',
 'config': {'output_hidden_states': True},
 'do_lower_case': False,
 'early_stopping_consider_epochs': False,
 'early_stopping_delta': 0,
 'early_stopping_metric': 'eval_loss',
 'early_stopping_metric_minimize': True,
 'early_stopping_patience': 3,
 'encoding': None,
 'eval_batch_size': 8,
 'evaluate_during_training': True,
 'evaluate_during_training_silent': True,
 'evaluate_during_training_steps': 2000,
 'evaluate_during_training_verbose': False,
 'fp16': False,
 'fp16_opt_level': 'O1',
 'gradient_accumulation_steps': 1,
 'learning_rate': 9.659e-05,
 'local_rank': -1,
 'logging_steps': 50,
 'manual_seed': 42,
 'max_grad_norm': 1.0,
 'max_seq_length': 300,
 '

In [66]:
trained_yield_bert.config.max_position_embeddings

512

### Work version

In [6]:
import torch
from rxnfp.models import SmilesClassificationModel, SmilesLanguageModelingModel
import pandas as pd
from rxn_yields.data import generate_buchwald_hartwig_rxns
from rdkit import Chem
from rdkit.Chem import DataStructs, AllChem
import numpy as np
from transformers import AutoTokenizer

print('Success')

Success


In [2]:
df = pd.read_excel('../Модели/yieldBERT/rxn_yields/data/Buchwald-Hartwig/Dreher_and_Doyle_input_data.xlsx', sheet_name='FullCV_01')
df['rxn'] = generate_buchwald_hartwig_rxns(df)

train_df = df.iloc[:2767][['rxn', 'Output']] 
test_df = df.iloc[2767:][['rxn', 'Output']] #

train_df.columns = ['text', 'labels']
test_df.columns = ['text', 'labels']
mean = train_df.labels.mean()
std = train_df.labels.std()
train_df['labels'] = (train_df['labels'] - mean) / std
test_df['labels'] = (test_df['labels'] - mean) / std
train_df.head()

,text,labels
0,Clc1ccccn1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2...,1.387974
1,Brc1ccccn1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2...,-0.796876
2,CCc1ccc(I)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccc...,-0.827835
3,FC(F)(F)c1ccc(Cl)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd...,-0.464841
4,COc1ccc(Cl)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2cc...,-1.186082


In [3]:
model_path = '../Модели/yieldBERT/rxn_yields/trained_models/buchwald_hartwig/FullCV_01_split_2768/checkpoint-1730-epoch-10'

In [4]:
trained_yield_bert = SmilesClassificationModel('bert', model_path,
                                               num_labels=1,
                                               args={"regression": True,
                                                    'config': {"output_hidden_states": True}},
                                               use_cuda=False,
                                              )

Setting 'max_len_single_sentence' is now deprecated. This value is automatically set up.
Setting 'max_len_sentences_pair' is now deprecated. This value is automatically set up.


In [7]:
tokenizer1 = AutoTokenizer.from_pretrained(model_path)

/home/vyacheslav/anaconda/anaconda3/envs/yields/lib/python3.6/site-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,


In [8]:
bert_inputs = tokenizer1.batch_encode_plus(str(test_df.head(1).labels.values),
                                           max_length=trained_yield_bert.config.max_position_embeddings,
                                           padding=True,
                                           truncation=True,
                                           pad_to_max_length=True,
                                           return_tensors='pt')

In [9]:
len(bert_inputs)

3

In [10]:
bert_inputs

{'input_ids': tensor([[12, 11, 13,  ...,  0,  0,  0],
        [12, 11, 13,  ...,  0,  0,  0],
        [12, 24, 13,  ...,  0,  0,  0],
        ...,
        [12, 43, 13,  ...,  0,  0,  0],
        [12, 98, 13,  ...,  0,  0,  0],
        [12, 11, 13,  ...,  0,  0,  0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [11]:
with torch.no_grad():
    output = trained_yield_bert.model(**bert_inputs)

In [12]:
len(output)

2

In [22]:
# [CLS] token embeddings in position 0
embeddings = output[0].squeeze().cpu().numpy().tolist()
embeddings

[0.672431230545044,
 0.672431230545044,
 0.8746748566627502,
 0.6140751242637634,
 0.5577840805053711,
 0.522050142288208,
 0.6576945781707764,
 0.6140751242637634,
 0.5635161995887756,
 0.5149366855621338,
 0.5635161995887756,
 0.672431230545044]

In [24]:
output['last_hidden_state']

TypeError: tuple indices must be integers or slices, not str

In [15]:
len(output[1])

13

In [16]:
len(output[1][1])

12

In [17]:
len(output[1][1][1])

512

In [18]:
len(output[1][1][1][1])

256

In [151]:
from sklearn.decomposition import PCA

In [156]:
pca = PCA(n_components=5)

In [ ]:
pca.fit(output)